In [1]:
"""
OBJETIVO : Definir y entrenar lso modelos. Ademas, recolectar los resultados para analizarlos mas adelante  
"""
Autor='Diego Paredes'

In [2]:
#Manejo de Datos
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import boxcox


#Machine learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import Callback

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix

#Librerias estandar (Extras)
import re
import os
import time
import random
from datetime import datetime
import pickle

In [3]:
"""
DEFINIMOS EL PATH DEL PROYECTO 
"""
with open('../../path_base.txt') as f:
    path_base = f.read()
path_base

'C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/FinalTesis/Tesis2-DiegoParedes'

In [4]:
"""
Variables generales
"""
path_imagenes = 'F:/GOES/'     

products = ['C13','C07','C08']
times   = ['10','20','30','40','50','00']


In [5]:
!python --version
print(tf. __version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

#Limitamos el GPU, en caso se necesite
gpus = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_virtual_device_configuration(gpus[0],
                                                        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])

Python 3.9.11
2.8.0
Num GPUs Available:  1


In [6]:
"""
Metodos para realizar el entrenamient - evaluacion del modelo
"""

'\nMetodos para realizar el entrenamient - evaluacion del modelo\n'

In [ ]:
"""
fileExcel = f'{path_base}/Archivos/Resultados/Clasificacion/RNN/Clasificacion-20220626_17/_Stats-Clasificacion-20220626_17.xlsx'
obj0 = pd.read_pickle(f'{path_base}/Archivos/Resultados/Clasificacion/RNN/Clasificacion-20220626_17/Test-0.csv')
obj1 = pd.read_pickle(f'{path_base}/Archivos/Resultados/Clasificacion/RNN/Clasificacion-20220626_17/Test-1.csv')

#0 =  Model_01_LSTM_clase_20220626_174443
#1 =  Model_01_LSTM_clase_20220626_182255
#obj
with pd.ExcelWriter(fileExcel, mode="a", engine="openpyxl", if_sheet_exists='overlay') as writer:                    
    tempDF = pd.DataFrame(obj1)
    
    tempDF.columns = ['loss', 'acc', 'TP', 'TN', 'FP','FN',
                    'val_loss','val_acc','val_TP','val_TN','val_FP','val_FN']
    tempDF.to_excel(writer,startrow=0,sheet_name='1-20220626_182255')
"""
a = 0

In [7]:
def getConv2D(p,run):    
    model = keras.Sequential()
    shape = (p['margen'][run],p['margen'][run],p['canales'][run])
        
    if p['normLayer'][run]:
        model.add(tf.keras.layers.Rescaling(1./65536))    
        
    model.add(tf.keras.layers.Conv2D(128, (3,3), input_shape=(p['margen'][run],p['margen'][run],p['canales'][run]),padding='same', activation='relu'))    
    
    for iConv in range(p['cnn_cant'][run]):
        units = p['cnn_units'][run][iConv]
        maxPool = p['maxPool'][run][iConv]
        droupout = p['droupout'][run][iConv]        
        
        model.add(tf.keras.layers.Conv2D(units, (3,3), padding='same', activation='relu'))
        if maxPool:
            model.add(tf.keras.layers.MaxPooling2D())
        if droupout:
            model.add(tf.keras.layers.Dropout(droupout))       
   
    
    model.add(tf.keras.layers.GlobalMaxPool2D())
    
    return model    

In [8]:
def crearModelo(p,run, redTipo):    
    convnet = getConv2D(p,run)
    shape = (p['tiempos'][run],p['margen'][run],p['margen'][run],p['canales'][run])    
    input_1 = tf.keras.layers.Input(shape=shape)
    timeD = tf.keras.layers.TimeDistributed(convnet)(input_1)
    
    units_rnn = p['rnn_units'][run]
    if p['rnn_tipo'][run] == 'LSTM':
        _rnn =  tf.keras.layers.LSTM(units_rnn)(timeD)
    elif p['rnn_tipo'][run] == 'SimpleRNN':
         _rnn =tf.keras.layers.SimpleRNN(units_rnn)(timeD)
    elif p['rnn_tipo'][run] == 'GRU':
         _rnn =tf.keras.layers.GRU(units_rnn)(timeD)
    else:
        print(f'ERROR : No se encontro el tipo de RNN: {p["rnn_tipo"]}')
        return None
    
    listConcat = [_rnn]
    listInputs = [input_1]
    
    if len(p['inputs'][run])>1:
        #Agregamos los otros atrbutos        
        for attr in p['inputs'][run][1:]:
            # The other input
            input_x = tf.keras.layers.Input(shape=(1,))
            listConcat.append(input_x)
            listInputs.append(input_x)

            
        # Concatenate
        final = tf.keras.layers.Concatenate()(listConcat) 
    
    
    
    dense_capas = [final]
    for iDense in range(p['dense_cant'][run]):
        units = p['dense_units'][run][iDense]
        dense_capas.append(tf.keras.layers.Dense(units=units, activation=tf.keras.activations.relu)(dense_capas[iDense]))
               
    
    # output
    if redTipo == 'Regresion':
        output = tf.keras.layers.Dense(units=1)(dense_capas[-1])      
    elif redTipo == 'Clasificacion':
        output = tf.keras.layers.Dense(units=1,activation=tf.keras.activations.sigmoid)(dense_capas[-1])
    else:
        print(f"No se pudo crear el modelo outputs no esta bien definido {p['redTipo']}")
        return -1      
    
    full_model = tf.keras.Model(inputs=listInputs, outputs=[output])

    return full_model

In [9]:
def getMetrics(params, HP, run):
    redTipo = params['redTipo']
    paciencia = params['paciencia']
    
    lr = HP['lr'][run]    
    
    if redTipo == 'Clasificacion':    
        optimizer = keras.optimizers.Adam(learning_rate=lr)   
        if HP['loss'][run] == 'binary_crossentropy':
            loss_fn= keras.losses.BinaryCrossentropy()
        
        train_acc_metric = keras.metrics.BinaryCrossentropy()
        val_acc_metric = keras.metrics.BinaryCrossentropy()
        
        if paciencia:
            early_stopping = keras.callbacks.EarlyStopping(monitor="val_acc", patience=paciencia, mode="max")  
 
        
        metrics = ['acc', keras.metrics.TruePositives(),
                         keras.metrics.TrueNegatives(),
                         keras.metrics.FalsePositives(),
                         keras.metrics.FalseNegatives()]
        

    elif redTipo == 'Regresion':
        optimizer = keras.optimizers.Adam(learning_rate=lr)
        loss_fn=keras.losses.MeanSquaredError()
        train_acc_metric = keras.metrics.MeanSquaredError()
        val_acc_metric = keras.metrics.MeanSquaredError()
        if paciencia:
            early_stopping = keras.callbacks.EarlyStopping(monitor="val_mse", patience=paciencia, mode="max")                                            
        metrics = ['mse']
        
        
    else:
        print('No se pudo crear las metricas')
        return -1    
         
        
    logs = Callback()
    callbacks = [logs]                     
    if paciencia:
        callbacks.append(early_stopping)
        
    metrics = {'optimizer': optimizer, 'loss_fn':loss_fn,'train_acc_metric': train_acc_metric,
               'val_acc_metric': val_acc_metric, 'metrics': metrics,'callbacks': callbacks}
    
    return metrics
        

In [10]:
def applyDA(img, DA):
    if DA == 1:
        return tf.image.flip_left_right(img)
    elif DA == 2:
        return tf.image.flip_up_down(img)
    elif DA == 3:
        img = tf.image.flip_left_right(img)
        return tf.image.flip_up_down(img)   
    else:
        return img

In [11]:
#Transformamos un filename tensor en una imagen
def read_png_file(item, value,p, run, path_base, DA=0, _3D=False, redTipo='Clasificacion'):
    # imagenData[0] = XO
    # imagenData[1] = XA
    # imagenData[2] = Fecha
    imagenData = tf.strings.split(item['imagen'], sep='--')
    
    
    size = int(p['margen'][run] / 2)
    timeJoin = []
    for j in range(p['tiempos'][run]-1,-1,-1):
        filename = path_base + 'PNG/' + imagenData[2] + '/' + imagenData[2] + '_' + str(j) + '.png'        
        image_string = tf.io.read_file(filename)
        img_decoded = tf.io.decode_png(image_string, dtype=tf.uint16, channels=3)       
        
        if DA:
            img_decoded = applyDA(img_decoded, item['DA'])
                
        timeJoin.insert(0,img_decoded[int(imagenData[1]) - size:int(imagenData[1]) + size,
                                      int(imagenData[0]) - size:int(imagenData[0]) + size,
                                      0:p['canales'][run]])
 
        
    if p['tiempos'][run]==1:
        imagenData = tf.reshape(timeJoin[0],(p['margen'][run],p['margen'][run],p['canales'][run]))
    else:
        if _3D:        
            img = tf.reduce_mean( timeJoin , axis=0 )
            imagenData = tf.reshape(img,(p['margen'][run],p['margen'][run],p['canales'][run]))
        else:
            img = tf.stack(timeJoin, axis=0)
            imagenData = tf.reshape(img,(p['tiempos'][run],p['margen'][run],p['margen'][run],p['canales'][run]))
        
    
    
    if len(p['inputs'][run]) == 1:
        return imagenData, int(value)
    
    item['imagen'] = imagenData
    itemL = []
    for inpL in p['inputs'][run]:
        itemL.append(item[inpL])
    
    if redTipo=='Regresion':
        return tuple(itemL), float(value)
    else:     
        return tuple(itemL), int(value)


In [12]:
def splitDataset(p,HP,run, path_imagenes):
    
    test = pd.read_csv(p['dsVal'])
    train = pd.read_csv(p['dsTrain'])
    
    if p['dataset']:
        train =  train.sample(frac=p['dataset'])
        test = test.sample(frac=p['dataset'])
    
               
    inputsList = {}
    inputsListTest = {}
    
    # Agregamos un atributo para indicar que el dato va realizar DA
    if p['DA']:        
        inputsList['DA'] = train['DA'].tolist() 
        
    print(f'Tamaño del dataset: Train {len(train)}  - Val {len(test)}') 
    
    for inp in HP['inputs'][run]:
        inputsList[inp] = train[inp].tolist()  
        inputsListTest[inp] = test[inp].tolist()  
        
       
        
    train_dataset = tf.data.Dataset.from_tensor_slices(((inputsList),train[HP['outputs'][run]].tolist()))           
    val_dataset = tf.data.Dataset.from_tensor_slices(((inputsListTest),test[HP['outputs'][run]].tolist()))     
    
    train_dataset = train_dataset.map(lambda x ,y : read_png_file(x,y,HP,run,path_imagenes,p['DA'],p['meanMatrizImagen'],p['redTipo']))
    val_dataset = val_dataset.map(lambda x ,y : read_png_file(x,y,HP,run,path_imagenes,False,p['meanMatrizImagen'],p['redTipo']))
       
    
    train_dataset = train_dataset.batch(p['batch'])
    val_dataset = val_dataset.batch(p['batch'])
    
    
    return train_dataset, val_dataset

In [13]:
def crearDir(path, newDir):
    try:
        pathT = os.path.join(path, newDir)
        os.mkdir(pathT)
        return pathT
    except FileExistsError:
        return pathT
        pass
    except:
        print(f"No se pudo crear el directorio: {newDir}")
        pritn(f'Path base: {path}')
        pritn(f'Nuevo    : {newDir}')        
        return None

In [14]:
def iniciarProyect(path_base, params, HP):
    repDir = crearDir(f'{path_base}/Archivos/Resultados', params["redTipo"])   
    repDir = crearDir(repDir, params["directory"])
    repDir = crearDir(repDir, params["Proyect"])
    
    excelFile = f'{repDir}/Stats-{params["Proyect"]}.xlsx'
    
    if params['record'] and not os.path.exists(excelFile):
        writer = pd.ExcelWriter(excelFile, engine = 'xlsxwriter')
        #joinParams = params | HP
        keys_values = params.items()
        strParams = {str(key): str(value) for key, value in keys_values}        
        
        pd.DataFrame(strParams,index=[0]).to_excel(writer, sheet_name = 'Informacion')
        
        pd.DataFrame(HP).to_excel(writer, sheet_name = 'Informacion',startrow=3)
        
        writer.save()     
    return repDir , excelFile

In [15]:
def getCM(logs):
    lKeys = list(logs.keys())
    
    try:
        TN = int(logs[[x for x in lKeys if 'val_true_negatives' in x][0]])
        TP = int(logs[[x for x in lKeys if 'val_true_positives' in x][0]])
        FN = int(logs[[x for x in lKeys if 'val_false_negatives' in x][0]])
        FP = int(logs[[x for x in lKeys if 'val_false_positives' in x][0]])
    except:
        print(f'\nNo se pudo leer keys para la matriz de confucion en logs : {lKeys}')
        print(f'Se intento leer: val_true_negatives,val_true_positives, val_false_negatives y val_false_positives')
    
       
    y_true =  [0]*TN + [1]*TP + [1]*FN + [0]*FP
    _y_pred = [0]*TN + [1]*TP + [0]*FN + [1]*FP
    
    return TN, FP, FN, TP, np.array(y_true), np.array(_y_pred)

In [16]:
class CustomCB(Callback):
    """ Custom callback to compute metrics at the end of each training epoch"""
    def __init__(self, val_ds=None, WANDB=True):     
        self.val_ds = val_ds  
        self.history = {}
        self.wandb = WANDB
   
   
    def on_epoch_end(self, epoch, logs={}):  
        TN, FP, FN, TP, y_true, _y_pred = getCM(logs)
        
        self.history.setdefault('epoch', []).append(epoch)
        
        self.history.setdefault('loss', []).append(logs['loss']) 
        self.history.setdefault('acc', []).append(logs['acc'])  
        self.history.setdefault('val_loss', []).append(logs['val_loss']) 
        self.history.setdefault('val_acc', []).append(logs['val_acc']) 
        
        
        self.history.setdefault('val_TN', []).append(TN) 
        self.history.setdefault('val_FP', []).append(FP)
        self.history.setdefault('val_FN', []).append(FN) 
        self.history.setdefault('val_TP', []).append(TP) 
        
            
        if self.wandb:
            wandb.log({"conf_mat" : wandb.plot.confusion_matrix(probs=None,
                                    preds=_y_pred, y_true=y_true,
                                    class_names=[0,1]),                   
                       'val_TN' :TN,'val_FN' :FN,'val_TP' :TP,'val_FP' :FP,
                       'val_acc': logs['val_acc'],'loss' : logs['loss'],
                       'val_loss': logs['val_loss'],'acc' : logs['acc']                  
                      })


In [17]:
def crearCallbacks(statsDir, HP,run, metricas, p):
    CB = metricas['callbacks']
    
    idModel = datetime.today().strftime("%Y%m%d_%H%M%S")
    checkpoint_path = statsDir + '/Model_{epoch:02d}_' + f'{HP["rnn_tipo"][run]}_{HP["outputs"][run]}_{idModel}.hdf5' 
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,verbose=1)
    
    # Iniciamos WANDB
    if p['record']:        
        CB.append(cp_callback)   
     
    if p['WANDB']:
        config = dict(learning_rate=HP['lr'][run], epochs = p['epocas'],
             batch_size = p['batch'],architecture="CNN", 
             num_classes = 2,)
        wandb.init(project=f'{p["Proyect"]}',            
                   config=config,
                   name= f'Ex_({HP["canales"][run]}-{HP["tiempos"][run]}-{HP["margen"][run]})_{idModel}')   
                                               
    return CB, idModel

In [18]:
def trainModel(path_base,path_imagenes,params,HP, criterio):        
    """ Creamos los directorios para los reportes  """             
    repDir, statsFile  = iniciarProyect(path_base, params, HP)
    print(f'DIRECTORIO BASE : {repDir}')
    
    
    """ Comenzamos el entrenamiento """ 
    # Una iteracion por cada Hiperparametro (HP) que existe
    ds_i = 0  
    resultados = [] 
    for run in range(HP['runs']):
        ds_i += 1                 
        print(f'Inicio de la prueba N°: {(run+1)}/{HP["runs"]}')         
        print(f'- Nombre del Proyecto : {params["Proyect"]}')
        print(f'- Batch size          : {params["batch"]}')
        print(f'- Criterio {criterio} : {HP[criterio][run]}')
        print('__________________________________________________')        
        
        """ LEEMOS EL DATASET A USAR  """     
        train_dataset, val_dataset = splitDataset(params,HP,run, path_imagenes)
        

        """ DEFINIMOS Y INICAMOS EL MODELO """
        model = crearModelo(HP,run,params['redTipo']) 
        metricas = getMetrics(params, HP, run)
        model.compile(optimizer=metricas['optimizer'],loss=metricas['loss_fn'],metrics=metricas['metrics'],)
        

        """ CALLBACKS """
        CB, idModel = crearCallbacks(repDir, HP,run, metricas, params)           
        if params['redTipo'] == 'Clasificacion':
            hist =  CustomCB(val_dataset, params['WANDB'])

            
        """ ENTRENAMIENTO """
        history = model.fit(train_dataset,batch_size=params['batch'],                            
                            epochs=params['epocas'],callbacks=CB,
                            validation_data=val_dataset,
                            validation_batch_size=params['batch'],
                            verbose=1)
        
        
        
        
        """ GUARDAMOS REPORTES """                
        
        
        try:
            resultados.append(history.history)
            # Guardamos las estadisticas
            if params['record']:                
                with pd.ExcelWriter(statsFile, mode="a", engine="openpyxl", if_sheet_exists='overlay') as writer:                    
                    tempDF = pd.DataFrame(history.history)
                    if params['redTipo'] == 'Clasificacion':
                        tempDF.columns = ['loss', 'acc', 'TP', 'TN', 'FP','FN',
                                          'val_loss','val_acc','val_TP','val_TN','val_FP','val_FN']
                    tempDF.to_excel(writer,startrow=0,
                                    sheet_name=f'{run}-{idModel}')
            if params['WANDB']:
                wandb.finish()            
        except:
            print(f'ERROR: No se pudo guardar los resulatdos test: {run}')
            try:
                with open(f'{repDir}/Test-{run}.csv', 'wb') as file_pi:
                    pickle.dump(history.history, file_pi)
            except Exception:
                print(traceback.format_exc())
                print(f'ERROR: No se pudo guardar el hist temporal')
                return history
            
        
    return resultados

In [19]:
#import wandb
#from wandb.keras import WandbCallback
#wandb.login()

In [37]:
%%time
"""
 Definimos las varibles para las iteraciones
"""

modelTipo = 'Clasificacion'
rnnTipo = 'LSTM'
idProject = datetime.today().strftime("%Y%m%d_%H")

p_train = {
            # Variables generales
          'products': products,
          'times'   : times,
    
            # Reportes
          'directory': 'RNN',
          'Proyect'  : f'{modelTipo}-{idProject}', # TesisDiego
          'record'   : True,  # Grabar los resultados en  excels    
          'WANDB'    : False, # Grabar los resultados en WANDB
    
            # Datos del modelo
          'redTipo'  : modelTipo, # Clasificacion / Regresion
          'rnn'      : True,  # Redes recurrentes          
          'meanMatrizImagen' : False, # !!! RNN modelos SIEMPRE EN FALSE  !!!
          
        
            # Variables del entrenamiento                
          'batch'    : 32,     
          'epocas'   : 40,  
          'paciencia': 15,   # 0 = No paciencia  
    
    
           # Dataset
          'dsTrain'  : f'{path_base}/Archivos/Dataset/{modelTipo}/Entrenamiento/SplitConDA_20C_DM_V2/CLASE_TrainDS_1.csv', 
          'dsVal'    : f'{path_base}/Archivos/Dataset/{modelTipo}/Validacion/ClaseV2_DUD_ValidacionDS.csv',
          'dataset'  : 1,     # 1 = 100% del ds
          'DA'       : True,  # Usaulmente para clasificacion          
         }

cantRuns = 6

 # Hiper parametros     
hiperparams = {    
               # General
              'dsTName'     :['SplitConDA_20C_DM_V2']*cantRuns,               
              'inputs'     : [['imagen', 'dato', 'altura']]*cantRuns,
              'outputs'    : ['clase']*cantRuns,
    
              # Modelo
              'lr'         : [0.001]*cantRuns, 
              'loss'       : ['binary_crossentropy']*cantRuns,
              'normLayer'  : [True]*cantRuns,
    
               # Capas convulucionales
              'cnn_cant'   : [2]*cantRuns,
              'cnn_units'  : [[64,32]]*cantRuns,
              'droupout'   : [[0.2,0.2]]*cantRuns,
              'maxPool'    : [[False,True]]*cantRuns,              
    
               # Capas Recurrentes
              'rnn_tipo'   : [rnnTipo]*cantRuns,
              'rnn_units'  : [64]*cantRuns,    
              
              # Capas densas
              'dense_cant' : [4]*cantRuns,
              'dense_units': [[64,32,32,16]]*cantRuns,
              
              # Imagenes satelitates
              'canales'    : [2]*cantRuns,
              'tiempos'    : [6]*cantRuns,
              'margen'     : [10,12,40,44,50,60],
    
              # -
              'runs'       : cantRuns
              }



CPU times: total: 0 ns
Wall time: 0 ns


In [38]:
# Forma del DS
if p_train['dsVal']:
    dfVal = pd.read_csv(p_train['dsVal'])
    print(f'VALDIACION : {len(dfVal)}')
    print('--------------------------------------------------')
    
tempDF = pd.read_csv(p_train['dsTrain'])
print(f'TRAIN: {len(tempDF)}')
tempDF.head(2)

VALDIACION : 3207
--------------------------------------------------
TRAIN: 6008


,Unnamed: 0.1,Unnamed: 0,nombre,codigo,XO,XA,longitud,latitud,altura,dato,...,99%,75%,umb1,umb2,fecha,flag,flagV2,imagen,clase,DA
0,0,38368,SANTA CLOTILDE,47271776,548,215,-73.67925,-2.48757,150.0,0.3,...,9.936,0.0,0.6,6.6,2021-10-04-08,M0000002,D01,548--215--2021-10-04-08,0,3
1,1,38369,INTIHUATANA H,472935F2,610,808,-72.56342,-13.17408,1778.0,0.2,...,0.000,0.0,3.1,7.5,2021-12-24-13,M0000002,C01,610--808--2021-12-24-13,0,3


In [39]:
resultado = trainModel(path_base,path_imagenes,p_train,hiperparams, 'tiempos')

DIRECTORIO BASE : C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/FinalTesis/Tesis2-DiegoParedes/Archivos/Resultados\Clasificacion\RNN\Clasificacion-20220627_10
Inicio de la prueba N°: 1/6
- Nombre del Proyecto : Clasificacion-20220627_10
- Batch size          : 32
- Criterio tiempos : 6
__________________________________________________
Tamaño del dataset: Train 6008  - Val 3207
Epoch 1/40
188/188 [==============================] - ETA: 0s - loss: 4.3478 - acc: 0.5824 - true_positives_11: 1667.0000 - true_negatives_11: 1832.0000 - false_positives_11: 1172.0000 - false_negatives_11: 1337.0000
Epoch 1: saving model to C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/FinalTesis/Tesis2-DiegoParedes/Archivos/Resultados\Clasificacion\RNN\Clasificacion-20220627_10\Model_01_LSTM_clase_20220627_102320.hdf5
188/188 [==============================] - 442s 2s/step - loss: 4.3478 - acc: 0.5824 - true_positives_11: 1667.0000 - true_negatives_11: 1832.0000 - false_positives_11: 1172.0000 - false_negatives_11: 13

Epoch 11/40
188/188 [==============================] - ETA: 0s - loss: 0.6829 - acc: 0.6701 - true_positives_11: 1705.0000 - true_negatives_11: 2321.0000 - false_positives_11: 683.0000 - false_negatives_11: 1299.0000
Epoch 11: saving model to C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/FinalTesis/Tesis2-DiegoParedes/Archivos/Resultados\Clasificacion\RNN\Clasificacion-20220627_10\Model_11_LSTM_clase_20220627_102320.hdf5
188/188 [==============================] - 599s 3s/step - loss: 0.6829 - acc: 0.6701 - true_positives_11: 1705.0000 - true_negatives_11: 2321.0000 - false_positives_11: 683.0000 - false_negatives_11: 1299.0000 - val_loss: 0.4992 - val_acc: 0.6779 - val_true_positives_11: 2139.0000 - val_true_negatives_11: 35.0000 - val_false_positives_11: 15.0000 - val_false_negatives_11: 1018.0000
Epoch 12/40
188/188 [==============================] - ETA: 0s - loss: 0.6589 - acc: 0.6819 - true_positives_11: 1731.0000 - true_negatives_11: 2366.0000 - false_positives_11: 638.0000 - false_n

188/188 [==============================] - ETA: 0s - loss: 0.5575 - acc: 0.7440 - true_positives_11: 2093.0000 - true_negatives_11: 2377.0000 - false_positives_11: 627.0000 - false_negatives_11: 911.0000
Epoch 21: saving model to C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/FinalTesis/Tesis2-DiegoParedes/Archivos/Resultados\Clasificacion\RNN\Clasificacion-20220627_10\Model_21_LSTM_clase_20220627_102320.hdf5
188/188 [==============================] - 491s 3s/step - loss: 0.5575 - acc: 0.7440 - true_positives_11: 2093.0000 - true_negatives_11: 2377.0000 - false_positives_11: 627.0000 - false_negatives_11: 911.0000 - val_loss: 0.4096 - val_acc: 0.7739 - val_true_positives_11: 2448.0000 - val_true_negatives_11: 34.0000 - val_false_positives_11: 16.0000 - val_false_negatives_11: 709.0000
Epoch 22/40
188/188 [==============================] - ETA: 0s - loss: 0.5570 - acc: 0.7423 - true_positives_11: 2074.0000 - true_negatives_11: 2386.0000 - false_positives_11: 618.0000 - false_negatives_11: 93

188/188 [==============================] - ETA: 0s - loss: 0.5561 - acc: 0.7437 - true_positives_11: 2079.0000 - true_negatives_11: 2389.0000 - false_positives_11: 615.0000 - false_negatives_11: 925.0000
Epoch 31: saving model to C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/FinalTesis/Tesis2-DiegoParedes/Archivos/Resultados\Clasificacion\RNN\Clasificacion-20220627_10\Model_31_LSTM_clase_20220627_102320.hdf5
188/188 [==============================] - 375s 2s/step - loss: 0.5561 - acc: 0.7437 - true_positives_11: 2079.0000 - true_negatives_11: 2389.0000 - false_positives_11: 615.0000 - false_negatives_11: 925.0000 - val_loss: 0.3920 - val_acc: 0.8120 - val_true_positives_11: 2572.0000 - val_true_negatives_11: 32.0000 - val_false_positives_11: 18.0000 - val_false_negatives_11: 585.0000
Epoch 32/40
188/188 [==============================] - ETA: 0s - loss: 0.5581 - acc: 0.7378 - true_positives_11: 2118.0000 - true_negatives_11: 2315.0000 - false_positives_11: 689.0000 - false_negatives_11: 88

Epoch 1/40
188/188 [==============================] - ETA: 0s - loss: 1.2743 - acc: 0.5867 - true_positives_12: 1659.0000 - true_negatives_12: 1866.0000 - false_positives_12: 1138.0000 - false_negatives_12: 1345.0000
Epoch 1: saving model to C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/FinalTesis/Tesis2-DiegoParedes/Archivos/Resultados\Clasificacion\RNN\Clasificacion-20220627_10\Model_01_LSTM_clase_20220627_151807.hdf5
188/188 [==============================] - 463s 2s/step - loss: 1.2743 - acc: 0.5867 - true_positives_12: 1659.0000 - true_negatives_12: 1866.0000 - false_positives_12: 1138.0000 - false_negatives_12: 1345.0000 - val_loss: 0.2969 - val_acc: 0.8207 - val_true_positives_12: 2619.0000 - val_true_negatives_12: 13.0000 - val_false_positives_12: 37.0000 - val_false_negatives_12: 538.0000
Epoch 2/40
188/188 [==============================] - ETA: 0s - loss: 1.1396 - acc: 0.6128 - true_positives_12: 1668.0000 - true_negatives_12: 2014.0000 - false_positives_12: 990.0000 - false_neg

...

188/188 [==============================] - ETA: 0s - loss: 0.5352 - acc: 0.7565 - true_positives_15: 2270.0000 - true_negatives_15: 2275.0000 - false_positives_15: 729.0000 - false_negatives_15: 734.0000
Epoch 31: saving model to C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/FinalTesis/Tesis2-DiegoParedes/Archivos/Resultados\Clasificacion\RNN\Clasificacion-20220627_10\Model_31_LSTM_clase_20220628_020513.hdf5
188/188 [==============================] - 328s 2s/step - loss: 0.5352 - acc: 0.7565 - true_positives_15: 2270.0000 - true_negatives_15: 2275.0000 - false_positives_15: 729.0000 - false_negatives_15: 734.0000 - val_loss: 0.5749 - val_acc: 0.6692 - val_true_positives_15: 2109.0000 - val_true_negatives_15: 37.0000 - val_false_positives_15: 13.0000 - val_false_negatives_15: 1048.0000
Epoch 32/40
188/188 [==============================] - ETA: 0s - loss: 0.5314 - acc: 0.7558 - true_positives_15: 2298.0000 - true_negatives_15: 2243.0000 - false_positives_15: 761.0000 - false_negatives_15: 7

Epoch 1/40
188/188 [==============================] - ETA: 0s - loss: 4.0703 - acc: 0.5995 - true_positives_16: 1801.0000 - true_negatives_16: 1801.0000 - false_positives_16: 1203.0000 - false_negatives_16: 1203.0000
Epoch 1: saving model to C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/FinalTesis/Tesis2-DiegoParedes/Archivos/Resultados\Clasificacion\RNN\Clasificacion-20220627_10\Model_01_LSTM_clase_20220628_061741.hdf5
188/188 [==============================] - 330s 2s/step - loss: 4.0703 - acc: 0.5995 - true_positives_16: 1801.0000 - true_negatives_16: 1801.0000 - false_positives_16: 1203.0000 - false_negatives_16: 1203.0000 - val_loss: 0.3746 - val_acc: 0.7162 - val_true_positives_16: 2267.0000 - val_true_negatives_16: 30.0000 - val_false_positives_16: 20.0000 - val_false_negatives_16: 890.0000
Epoch 2/40
188/188 [==============================] - ETA: 0s - loss: 0.8291 - acc: 0.6365 - true_positives_16: 1679.0000 - true_negatives_16: 2145.0000 - false_positives_16: 859.0000 - false_neg

188/188 [==============================] - ETA: 0s - loss: 0.5916 - acc: 0.7130 - true_positives_16: 1995.0000 - true_negatives_16: 2289.0000 - false_positives_16: 715.0000 - false_negatives_16: 1009.0000
Epoch 11: saving model to C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/FinalTesis/Tesis2-DiegoParedes/Archivos/Resultados\Clasificacion\RNN\Clasificacion-20220627_10\Model_11_LSTM_clase_20220628_061741.hdf5
188/188 [==============================] - 328s 2s/step - loss: 0.5916 - acc: 0.7130 - true_positives_16: 1995.0000 - true_negatives_16: 2289.0000 - false_positives_16: 715.0000 - false_negatives_16: 1009.0000 - val_loss: 0.4224 - val_acc: 0.7680 - val_true_positives_16: 2430.0000 - val_true_negatives_16: 33.0000 - val_false_positives_16: 17.0000 - val_false_negatives_16: 727.0000
Epoch 12/40
188/188 [==============================] - ETA: 0s - loss: 0.5817 - acc: 0.7190 - true_positives_16: 2041.0000 - true_negatives_16: 2279.0000 - false_positives_16: 725.0000 - false_negatives_16: 

188/188 [==============================] - ETA: 0s - loss: 0.5553 - acc: 0.7422 - true_positives_16: 2137.0000 - true_negatives_16: 2322.0000 - false_positives_16: 682.0000 - false_negatives_16: 867.0000
Epoch 21: saving model to C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/FinalTesis/Tesis2-DiegoParedes/Archivos/Resultados\Clasificacion\RNN\Clasificacion-20220627_10\Model_21_LSTM_clase_20220628_061741.hdf5
188/188 [==============================] - 544s 3s/step - loss: 0.5553 - acc: 0.7422 - true_positives_16: 2137.0000 - true_negatives_16: 2322.0000 - false_positives_16: 682.0000 - false_negatives_16: 867.0000 - val_loss: 0.5343 - val_acc: 0.6904 - val_true_positives_16: 2180.0000 - val_true_negatives_16: 34.0000 - val_false_positives_16: 16.0000 - val_false_negatives_16: 977.0000
Epoch 22/40
188/188 [==============================] - ETA: 0s - loss: 0.5995 - acc: 0.7047 - true_positives_16: 2005.0000 - true_negatives_16: 2229.0000 - false_positives_16: 775.0000 - false_negatives_16: 99

188/188 [==============================] - ETA: 0s - loss: 0.5474 - acc: 0.7475 - true_positives_16: 2112.0000 - true_negatives_16: 2379.0000 - false_positives_16: 625.0000 - false_negatives_16: 892.0000
Epoch 31: saving model to C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/FinalTesis/Tesis2-DiegoParedes/Archivos/Resultados\Clasificacion\RNN\Clasificacion-20220627_10\Model_31_LSTM_clase_20220628_061741.hdf5
188/188 [==============================] - 541s 3s/step - loss: 0.5474 - acc: 0.7475 - true_positives_16: 2112.0000 - true_negatives_16: 2379.0000 - false_positives_16: 625.0000 - false_negatives_16: 892.0000 - val_loss: 0.5423 - val_acc: 0.6913 - val_true_positives_16: 2180.0000 - val_true_negatives_16: 37.0000 - val_false_positives_16: 13.0000 - val_false_negatives_16: 977.0000
Epoch 32/40
188/188 [==============================] - ETA: 0s - loss: 0.5364 - acc: 0.7495 - true_positives_16: 2191.0000 - true_negatives_16: 2312.0000 - false_positives_16: 692.0000 - false_negatives_16: 81

In [ ]:
%%time
"""
 Definimos las varibles para las iteraciones
"""

modelTipo = 'Clasificacion'
rnnTipo = 'LSTM'
idProject = datetime.today().strftime("%Y%m%d_%H")

p_train = {
            # Variables generales
          'products': products,
          'times'   : times,
    
            # Reportes
          'directory': 'RNN',
          'Proyect'  : f'{modelTipo}-{idProject}', # TesisDiego
          'record'   : True,  # Grabar los resultados en  excels    
          'WANDB'    : False, # Grabar los resultados en WANDB
    
            # Datos del modelo
          'redTipo'  : modelTipo, # Clasificacion / Regresion
          'rnn'      : True,  # Redes recurrentes          
          'meanMatrizImagen' : False, # !!! RNN modelos SIEMPRE EN FALSE  !!!
          
        
            # Variables del entrenamiento                
          'batch'    : 32,     
          'epocas'   : 40,  
          'paciencia': 15,   # 0 = No paciencia  
    
    
           # Dataset
          'dsTrain'  : f'{path_base}/Archivos/Dataset/{modelTipo}/Entrenamiento/SplitConDA_20C_DM_V2/CLASE_TrainDS_1.csv', 
          'dsVal'    : f'{path_base}/Archivos/Dataset/{modelTipo}/Validacion/ClaseV2_DUD_ValidacionDS.csv',
          'dataset'  : 1,     # 1 = 100% del ds
          'DA'       : True,  # Usaulmente para clasificacion          
         }

cantRuns = 6

 # Hiper parametros     
hiperparams = {    
               # General
              'dsTName'     :['SplitConDA_20C_DM_V2']*cantRuns,               
              'inputs'     : [['imagen', 'dato','umb1','altura']]*cantRuns,
              'outputs'    : ['clase']*cantRuns,
    
              # Modelo
              'lr'         : [0.001]*cantRuns, 
              'loss'       : ['binary_crossentropy']*cantRuns,
              'normLayer'  : [True]*cantRuns,
    
               # Capas convulucionales
              'cnn_cant'   : [2]*cantRuns,
              'cnn_units'  : [[64,32]]*cantRuns,
              'droupout'   : [[0.2,0.2]]*cantRuns,
              'maxPool'    : [[False,True]]*cantRuns,              
    
               # Capas Recurrentes
              'rnn_tipo'   : [rnnTipo]*cantRuns,
              'rnn_units'  : [64]*cantRuns,    
              
              # Capas densas
              'dense_cant' : [4]*cantRuns,
              'dense_units': [[64,32,32,16]]*cantRuns,
              
              # Imagenes satelitates
              'canales'    : [2]*cantRuns,
              'tiempos'    : [6]*cantRuns,
              'margen'     : [10,12,40,44,50,60],
    
              # -
              'runs'       : cantRuns
              }

resultado = trainModel(path_base,path_imagenes,p_train,hiperparams, 'margen')

DIRECTORIO BASE : C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/FinalTesis/Tesis2-DiegoParedes/Archivos/Resultados\Clasificacion\RNN\Clasificacion-20220628_10
Inicio de la prueba N°: 1/6
- Nombre del Proyecto : Clasificacion-20220628_10
- Batch size          : 32
- Criterio margen : 10
__________________________________________________
Tamaño del dataset: Train 6008  - Val 3207
Epoch 1/40
188/188 [==============================] - ETA: 0s - loss: 1.6719 - acc: 0.6167 - true_positives_17: 1750.0000 - true_negatives_17: 1955.0000 - false_positives_17: 1049.0000 - false_negatives_17: 1254.0000
Epoch 1: saving model to C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/FinalTesis/Tesis2-DiegoParedes/Archivos/Resultados\Clasificacion\RNN\Clasificacion-20220628_10\Model_01_LSTM_clase_20220628_101231.hdf5
188/188 [==============================] - 328s 2s/step - loss: 1.6719 - acc: 0.6167 - true_positives_17: 1750.0000 - true_negatives_17: 1955.0000 - false_positives_17: 1049.0000 - false_negatives_17: 12

Epoch 11/40
188/188 [==============================] - ETA: 0s - loss: 0.6112 - acc: 0.6974 - true_positives_17: 1880.0000 - true_negatives_17: 2310.0000 - false_positives_17: 694.0000 - false_negatives_17: 1124.0000
Epoch 11: saving model to C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/FinalTesis/Tesis2-DiegoParedes/Archivos/Resultados\Clasificacion\RNN\Clasificacion-20220628_10\Model_11_LSTM_clase_20220628_101231.hdf5
188/188 [==============================] - 337s 2s/step - loss: 0.6112 - acc: 0.6974 - true_positives_17: 1880.0000 - true_negatives_17: 2310.0000 - false_positives_17: 694.0000 - false_negatives_17: 1124.0000 - val_loss: 1.1933 - val_acc: 0.0552 - val_true_positives_17: 128.0000 - val_true_negatives_17: 49.0000 - val_false_positives_17: 1.0000 - val_false_negatives_17: 3029.0000
Epoch 12/40
188/188 [==============================] - ETA: 0s - loss: 0.5974 - acc: 0.6972 - true_positives_17: 1895.0000 - true_negatives_17: 2294.0000 - false_positives_17: 710.0000 - false_neg

188/188 [==============================] - ETA: 0s - loss: 0.5365 - acc: 0.7447 - true_positives_17: 2141.0000 - true_negatives_17: 2333.0000 - false_positives_17: 671.0000 - false_negatives_17: 863.0000
Epoch 21: saving model to C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/FinalTesis/Tesis2-DiegoParedes/Archivos/Resultados\Clasificacion\RNN\Clasificacion-20220628_10\Model_21_LSTM_clase_20220628_101231.hdf5
188/188 [==============================] - 362s 2s/step - loss: 0.5365 - acc: 0.7447 - true_positives_17: 2141.0000 - true_negatives_17: 2333.0000 - false_positives_17: 671.0000 - false_negatives_17: 863.0000 - val_loss: 0.4383 - val_acc: 0.7558 - val_true_positives_17: 2390.0000 - val_true_negatives_17: 34.0000 - val_false_positives_17: 16.0000 - val_false_negatives_17: 767.0000
Epoch 22/40
188/188 [==============================] - ETA: 0s - loss: 0.5705 - acc: 0.7342 - true_positives_17: 2081.0000 - true_negatives_17: 2330.0000 - false_positives_17: 674.0000 - false_negatives_17: 92

188/188 [==============================] - ETA: 0s - loss: 0.5081 - acc: 0.7620 - true_positives_17: 2280.0000 - true_negatives_17: 2298.0000 - false_positives_17: 706.0000 - false_negatives_17: 724.0000
Epoch 31: saving model to C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/FinalTesis/Tesis2-DiegoParedes/Archivos/Resultados\Clasificacion\RNN\Clasificacion-20220628_10\Model_31_LSTM_clase_20220628_101231.hdf5
188/188 [==============================] - 530s 3s/step - loss: 0.5081 - acc: 0.7620 - true_positives_17: 2280.0000 - true_negatives_17: 2298.0000 - false_positives_17: 706.0000 - false_negatives_17: 724.0000 - val_loss: 0.5003 - val_acc: 0.7552 - val_true_positives_17: 2388.0000 - val_true_negatives_17: 34.0000 - val_false_positives_17: 16.0000 - val_false_negatives_17: 769.0000
Epoch 32/40
188/188 [==============================] - ETA: 0s - loss: 0.5082 - acc: 0.7602 - true_positives_17: 2236.0000 - true_negatives_17: 2331.0000 - false_positives_17: 673.0000 - false_negatives_17: 76

Tamaño del dataset: Train 6008  - Val 3207
Epoch 1/40
188/188 [==============================] - ETA: 0s - loss: 1.3241 - acc: 0.6029 - true_positives_18: 1713.0000 - true_negatives_18: 1909.0000 - false_positives_18: 1095.0000 - false_negatives_18: 1291.0000
Epoch 1: saving model to C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/FinalTesis/Tesis2-DiegoParedes/Archivos/Resultados\Clasificacion\RNN\Clasificacion-20220628_10\Model_01_LSTM_clase_20220628_143817.hdf5
188/188 [==============================] - 351s 2s/step - loss: 1.3241 - acc: 0.6029 - true_positives_18: 1713.0000 - true_negatives_18: 1909.0000 - false_positives_18: 1095.0000 - false_negatives_18: 1291.0000 - val_loss: 0.5937 - val_acc: 0.6399 - val_true_positives_18: 2016.0000 - val_true_negatives_18: 36.0000 - val_false_positives_18: 14.0000 - val_false_negatives_18: 1141.0000
Epoch 2/40
188/188 [==============================] - ETA: 0s - loss: 0.8537 - acc: 0.6410 - true_positives_18: 1663.0000 - true_negatives_18: 2188.000

120/188 [==================>...........] - ETA: 1:20 - loss: 0.5963 - acc: 0.7148 - true_positives_18: 1257.0000 - true_negatives_18: 1488.0000 - false_positives_18: 422.0000 - false_negatives_18: 673.0000